In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tifffile as tif
import os
from PIL import Image
import cv2
from tqdm import tqdm
import pickle

from sklearn.model_selection import StratifiedShuffleSplit
import albumentations as albu
from skimage.transform import resize
from pylab import rcParams
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from tensorflow.keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dropout, Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam

%matplotlib inline


In [2]:
height = 224
width = 224
channels = 3
input_shape = (height, width, channels)
INP_SHAPE = (height, width)
n_classes = 2
epochs = 15
batch_size = 64

In [3]:
def find_path(file,working =True):
    import sys
  
  
#     sys.stdout.write("Options for 'working aruguments [True,org,stripai]'")
#     sys.stdout.flush()
    print("Options for 'working aruguments [True,org,stripai]'",flush=False)

    if working ==True:
        for i in os.walk('../test/'):
            for j in i[2]:
                if j ==file:
                    return os.path.join(i[0],j)
    elif working =='org':
        for i in os.walk('../input/mayo-clinic-strip-ai'):
            for j in i[2]:
                if j ==file:
                    return os.path.join(i[0],j)
    elif working =='stripai':
        for i in os.walk('../input/stripai/archive'):
            for j in i[2]:
                if j ==file:
                    return os.path.join(i[0],j)

In [4]:
def resize_img(img, shape):
    return cv2.resize(img, (shape[1], shape[0]), interpolation=cv2.INTER_CUBIC)

In [5]:
height = 224
width = 224
channels = 3
input_shape = (height, width, channels)
INP_SHAPE = (height, width)
n_classes = 2
epochs = 15
batch_size = 64

In [6]:
class DataGenerator(keras.utils.Sequence):
    def __init__(self, images, labels=None, mode='fit', batch_size=batch_size, dim=(height, width), channels=channels, n_classes=n_classes, shuffle=True, augment=False):
        
        #initializing the configuration of the generator
        self.images = images
        self.labels = labels
        self.mode = mode
        self.batch_size = batch_size
        self.dim = dim
        self.channels = channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.augment = augment
        self.on_epoch_end()
   
    #method to be called after every epoch
    def on_epoch_end(self):
        self.indexes = np.arange(self.images.shape[0])
        if self.shuffle == True:
            np.random.shuffle(self.indexes)
    
    #return numbers of steps in an epoch using samples & batch size
    def __len__(self):
        return int(np.floor(len(self.images) / self.batch_size))
    
    #this method is called with the batch number as an argument to #obtain a given batch of data
    def __getitem__(self, index):
        #generate one batch of data
        #generate indexes of batch
        batch_indexes = self.indexes[index * self.batch_size:(index+1) * self.batch_size]
        
        #generate mini-batch of X
        X = np.empty((self.batch_size, *self.dim, self.channels))
        for i, ID in enumerate(batch_indexes):
            #generate pre-processed image
            img = self.images[ID]
            #image rescaling
            img = img.astype(np.float32)/255.
            #resizing as per new dimensions
#             img = resize_img(img, self.dim)
            X[i] = img
            
        #generate mini-batch of y
        if self.mode == 'fit':
            y = self.labels[batch_indexes]
            
            #augmentation on the training dataset
            if self.augment == True:
                X = self.__augment_batch(X)
            return X, y
        
        elif self.mode == 'predict':
            return X
        
        else:
            raise AttributeError("The mode should be set to either 'fit' or 'predict'.")
            
    #augmentation for one image
    def __random_transform(self, img):
        composition = albu.Compose([albu.HorizontalFlip(p=0.5),
                                   albu.VerticalFlip(p=0.5),
                                   albu.GridDistortion(p=0.2),
                                   albu.ElasticTransform(p=0.2)])
        return composition(image=img)['image']
    
    #augmentation for batch of images
    def __augment_batch(self, img_batch):
        for i in range(img_batch.shape[0]):
            img_batch[i] = self.__random_transform(img_batch[i])
        return img_batch

In [7]:
def ImgDataset(df,tif_ = False):
    train = 'label' in df.columns
    index = df.index
    img = []
    for image_id in tqdm(df['image_id']):
        if tif_:
#             print(find_path(image_id + ".tif",'org'))
            image = tif.imread(find_path(image_id + ".tif",'org'))
        else:
#             print(find_path(image_id + ".jpg",'stripai'))

            image = cv2.imread(find_path(image_id + ".jpg",'stripai'))

        label = 0
        
        image = resize_img(image, INP_SHAPE)
            
        if(train):
            label = df.iloc[index].label.replace({"CE" : 0, "LAA": 1}).tolist()
        patient_id = df.iloc[index].patient_id
        img.append(image)
    return np.asarray(img), np.asarray(label), np.asarray(patient_id.tolist())

In [8]:
train_df = pd.read_csv(find_path("train.csv",'org'))
test_df =  pd.read_csv("../input/mayo-clinic-strip-ai/test.csv")

Options for 'working aruguments [True,org,stripai]'


In [9]:
ImgSize= height
dirs = "../input/mayo-clinic-strip-ai/test/"
os.makedirs("../test/",exist_ok = True)
for i in tqdm(range(test_df.shape[0])):
    img_id = test_df.iloc[i].image_id
    try:
        sz = os.path.getsize(os.path.join(dirs,img_id + ".tif"))
    except:
        sz = 1000000000
    print(sz)
    Try = ""
    if(sz > 8e8):
        img = np.zeros((ImgSize,ImgSize,3), np.uint8)
    else:
        try:
            img = cv2.resize(tif.imread(os.path.join(dirs,img_id + ".tif")), (ImgSize, ImgSize))/255.
            Try = "True "+f"{img_id}"
        except:
            img = np.zeros((ImgSize,ImgSize,3), np.uint8)
            Try = "False"
            
    print(Try)
    cv2.imwrite(f"../test/{img_id}.jpg", img)
    del img
    import gc
    gc.collect()

  0%|          | 0/4 [00:00<?, ?it/s]

1312937514



 25%|██▌       | 1/4 [00:00<00:00,  5.74it/s]

109571430


 50%|█████     | 2/4 [00:04<00:05,  2.64s/it]

True 008e5c_0
351760030


 75%|███████▌  | 3/4 [00:17<00:07,  7.17s/it]

True 00c058_0
679166312


100%|██████████| 4/4 [00:39<00:00,  9.96s/it]

True 01adc5_0


In [10]:
xtrain,ytrain,pat_train = ImgDataset(train_df,False)
test,_,pat_test = ImgDataset(test_df,False)


  1%|          | 5/754 [00:00<00:30, 24.93it/s]

Options for 'working aruguments [True,org,stripai]'
Options for 'working aruguments [True,org,stripai]'
Options for 'working aruguments [True,org,stripai]'
Options for 'working aruguments [True,org,stripai]'
Options for 'working aruguments [True,org,stripai]'


  2%|▏         | 12/754 [00:00<00:18, 41.03it/s]

Options for 'working aruguments [True,org,stripai]'
Options for 'working aruguments [True,org,stripai]'
Options for 'working aruguments [True,org,stripai]'
Options for 'working aruguments [True,org,stripai]'
Options for 'working aruguments [True,org,stripai]'
Options for 'working aruguments [True,org,stripai]'
Options for 'working aruguments [True,org,stripai]'
Options for 'working aruguments [True,org,stripai]'
Options for 'working aruguments [True,org,stripai]'
Options for 'working aruguments [True,org,stripai]'
Options for 'working aruguments [True,org,stripai]'
Options for 'working aruguments [True,org,stripai]'
Options for 'working aruguments [True,org,stripai]'


  3%|▎         | 24/754 [00:00<00:14, 50.51it/s]

...

Options for 'working aruguments [True,org,stripai]'
Options for 'working aruguments [True,org,stripai]'
Options for 'working aruguments [True,org,stripai]'
Options for 'working aruguments [True,org,stripai]'
Options for 'working aruguments [True,org,stripai]'
Options for 'working aruguments [True,org,stripai]'
Options for 'working aruguments [True,org,stripai]'
Options for 'working aruguments [True,org,stripai]'
Options for 'working aruguments [True,org,stripai]'
Options for 'working aruguments [True,org,stripai]'
Options for 'working aruguments [True,org,stripai]'
Options for 'working aruguments [True,org,stripai]'
Options for 'working aruguments [True,org,stripai]'
Options for 'working aruguments [True,org,stripai]'
Options for 'working aruguments [True,org,stripai]'
Options for 'working aruguments [True,org,stripai]'
Options for 'working aruguments [True,org,stripai]'


100%|██████████| 754/754 [00:09<00:00, 76.59it/s]


Options for 'working aruguments [True,org,stripai]'
Options for 'working aruguments [True,org,stripai]'
Options for 'working aruguments [True,org,stripai]'


100%|██████████| 4/4 [00:00<00:00, 162.32it/s]

Options for 'working aruguments [True,org,stripai]'
Options for 'working aruguments [True,org,stripai]'
Options for 'working aruguments [True,org,stripai]'
Options for 'working aruguments [True,org,stripai]'


In [11]:
y_train_lb = pd.get_dummies(ytrain.astype(str)).values

In [12]:
# with open('xtrain((224, 224, 3)).pkl', 'wb') as file1:
      
#     # A new file will be created
#     pickle.dump(xtrain, file1)
    
# with open('ytrain((224, 224, 3)).pkl', 'wb') as file2:
      
#     # A new file will be created
#     pickle.dump(ytrain, file2)
# with open('pat_train((224, 224, 3)).pkl', 'wb') as file3:
      
#     # A new file will be created
#     pickle.dump(pat_train, file3)
    
    
# with open("./xtrain((224, 224, 3)).pkl", 'rb') as f1:
#     xtrain = pickle.load(f1)
#     f1.close()
# with open("./ytrain((224, 224, 3)).pkl", 'rb') as f2:
#     ytrain = pickle.load(f2)
#     f2.close()
    
# with open("./pat_train((224, 224, 3)).pkl", 'rb') as f3:
#     pat_train = pickle.load(f3)
#     f3.close()


In [13]:
xtrain.shape,ytrain.shape,pat_train.shape

((754, 224, 224, 3), (754,), (754,))

In [14]:
class_weights = {np.unique(ytrain,return_counts = True)[0][0]:(np.unique(ytrain,return_counts = True)[1][0].sum()/np.unique(ytrain,return_counts = True)[1].sum()),
                 np.unique(ytrain,return_counts = True)[0][1]:(np.unique(ytrain,return_counts = True)[1][1]/np.unique(ytrain,return_counts = True)[1].sum())
                }
class_weights

{0: 0.7254641909814323, 1: 0.27453580901856767}

In [15]:
# def CNN():

#   input_layer = Input(shape=(xtrain.shape[1],xtrain.shape[2],xtrain.shape[3]))
#   #x = Embedding(max_features, embed_size, weights=[embedding_matrix], trainable=False)(inp)
#   x = Conv2D(filters=32, kernel_size=3, padding='same', activation='relu')(input_layer) 
#   x = MaxPooling2D(pool_size=2)(x)
#   x = Conv2D(filters=64, kernel_size=3, padding='same', activation='relu')(x)
#   x = MaxPooling2D(pool_size=2)(x)
#   x = Conv2D(filters=64, kernel_size=2, padding='same', activation='relu')(x)
#   x = MaxPooling2D(pool_size=2)(x)
#   # x = GRU(16)(x)
#   # x = Dropout(0.3)(x)
#   x = Flatten()(x)
#   x = Dense(128, activation="relu")(x)
#   x = Dense(2,activation = 'sigmoid')(x)
#   model = Model(inputs=input_layer, outputs=x)
#   return model

# def tl_vgg():
#     inp = VGG16(include_top=False,input_shape = (INP_SHAPE+(3,)),weights=find_path("vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5"))
    
# #     inp = models.load_model(find_path('vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5'))
#     for layer in inp.layers:
#         layer.trainable = False

#     #flatten the last layer and add a fully connected layer as output
#     hidden = Flatten()(inp.output)
#     outputs = Dense(2, activation='sigmoid')(hidden)

#     #create the model
#     model = Model(inputs=inp.input, outputs=outputs)

#     #check the model architecture
#     model.summary()
#     return model
# def tl_IncepRes():
#     inp = tf.keras.applications.InceptionResNetV2(include_top=False,input_shape = (INP_SHAPE+(3,)),weights=find_path("inception_resnet_v2_weights_tf_dim_ordering_tf_kernels_notop.h5"))
    
# #     inp = models.load_model(find_path('vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5'))
#     for layer in inp.layers:
#         layer.trainable = False

#     #flatten the last layer and add a fully connected layer as output
#     hidden = Flatten()(inp.output)
#     outputs = Dense(2, activation='sigmoid')(hidden)

#     #create the model
#     model = Model(inputs=inp.input, outputs=outputs)

#     #check the model architecture
#     model.summary()
#     return model


In [16]:

def tl_efn():
    efnb0 = tf.keras.applications.EfficientNetB0(weights='../input/stripai/efficientnetb0_notop.h5', include_top=False,input_shape=input_shape, classes=n_classes)

    model = Sequential()
    model.add(efnb0)
    model.add(GlobalAveragePooling2D())
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='sigmoid'))

    model.summary()
    return model

In [17]:
# tl_efn()

In [18]:
from sklearn.model_selection import KFold
splits =StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=123)

In [19]:

for train_index, val_index in splits.split(xtrain, ytrain):
    X_train_data, X_val_data = xtrain[train_index], xtrain[val_index]
    y_train_data, y_val_data = y_train_lb[train_index], y_train_lb[val_index]

print("Number of training samples: ", X_train_data.shape[0])
print("Number of validation samples: ", X_val_data.shape[0])

Number of training samples:  603
Number of validation samples:  151


In [20]:
train_data_generator = DataGenerator(X_train_data, y_train_data, augment=True) 
valid_data_generator = DataGenerator(X_val_data, y_val_data, augment=False)

In [21]:
# from keras import backend as K
# import tensorflow as tf

# def KerasFocalLoss(target, input):
    
#     gamma = 2.
#     input = tf.cast(input, tf.float32)
    
#     max_val = K.clip(-input, 0, 1)
#     loss = input - input * target + max_val + K.log(K.exp(-max_val) + K.exp(-input - max_val))
#     invprobs = tf.log_sigmoid(-input * (target * 2.0 - 1.0))
#     loss = K.exp(invprobs * gamma) * loss
    
#     return K.mean(K.sum(loss, axis=1))

In [22]:
def train(models,trainx = xtrain,trainy=y_train_lb,fold = True):
    test_pred = np.zeros((test.shape[0],2))
    model = models()
    model.compile(optimizer = Adam(lr=0.0001),  loss='binary_crossentropy',metrics = ['accuracy'])

    #early stopping to monitor the validation loss and avoid overfitting
    early_stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10, restore_best_weights=True)

    #reducing learning rate on plateau
    rlrop = ReduceLROnPlateau(monitor='val_loss', mode='min', patience= 5, factor= 0.5, min_lr= 1e-6, verbose=1)
    
    if fold :
        avg = []
        for i ,(train_ind,test_ind) in enumerate(splits.split(trainx)):
            print(f"| Fold {i+1} |".center(100,"*"))
            x_train,x_test = trainx[train_ind],trainx[test_ind]

            y_train,y_test  = trainy[train_ind],trainy[test_ind]
    #         idg.fit((x_train,y_train))
            model.fit(x_train,y_train,callbacks = [early_stop, rlrop],class_weight = class_weights,epochs = 25,validation_data = (x_test,y_test))
    #         avg.append(model.predict(x_test))

            test_pred += model.predict(test)/5
    else:
        model.fit(train_data_generator,callbacks = [early_stop, rlrop],
                  steps_per_epoch = int( np.ceil(X_train_data.shape[0] / batch_size)),
                  class_weight = class_weights,
                  epochs = 25,
                  validation_data = valid_data_generator,
                validation_steps = int( np.ceil(X_val_data.shape[0] / batch_size)),
                    )
        test_pred = model.predict(test)
    
    return model,test_pred

In [23]:
# vgg,vgg_pred = train(tl_vgg)

In [24]:
# (vgg_pred)

In [25]:
# IncepRes,incep_pred = train(tl_IncepRes)


In [26]:
# incep_pred

In [27]:
# PRED = model.predict(test)
# PRED

In [28]:
# y_train_pred = model.predict(xtrain)

In [29]:
model_efn,efn_pred = train(tl_efn,fold = False)

2022-07-31 15:54:12.441747: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-31 15:54:12.595531: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-31 15:54:12.596306: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-31 15:54:12.597412: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficientnetb0 (Functional)  (None, 7, 7, 1280)        4049571   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1280)              0         
_________________________________________________________________
dropout (Dropout)            (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 2)                 2562      
Total params: 4,052,133
Trainable params: 4,010,110
Non-trainable params: 42,023
_________________________________________________________________


/opt/conda/lib/python3.7/site-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")
2022-07-31 15:54:18.854403: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/25


2022-07-31 15:54:28.712865: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


10/10 [==============================] - 23s 702ms/step - loss: 0.4143 - accuracy: 0.5295 - val_loss: 0.7221 - val_accuracy: 0.2734


In [30]:
print(efn_pred[:,0])
print(efn_pred[:,1][0])

[0.6890395  0.70542765 0.7319453  0.7183242 ]
0.442307


In [31]:
from sklearn.metrics import log_loss
# log_loss(ytrain.astype(int),y_train_pred)

In [32]:
from sklearn.metrics import accuracy_score

In [33]:
# IncepRes.save("(stripAi)IncepRes(150,150,3).h5")

In [34]:
# PRED = IncepRes.predict(test)
# PRED

In [35]:
def Predict(pred1,pred2 = 0):
    from IPython.display import display
#     model1 = models[0]
#     model2 = models[1]
    
#     pred1 = model1.predict(test) 
#     pred2 = model2.predict(test) 
#     print(np.array([pred1,pred2]))
    pred =pred1#( pred1*0.5) +(pred2*0.5)
    print(pred)
    prob = pd.DataFrame({"CE" : pred[:,0], "LAA" : pred[:,1], "id" : pat_test}).groupby("id").mean()
    submission = pd.read_csv("../input/mayo-clinic-strip-ai/sample_submission.csv")
    submission.CE = prob.CE.to_list()
    submission.LAA = prob.LAA.to_list()
    submission.patient_id = prob.index
    submission.to_csv("submission.csv", index = False)
    display(submission)

In [36]:
Predict(efn_pred)

[[0.6890395  0.442307  ]
 [0.70542765 0.5803662 ]
 [0.7319453  0.6975927 ]
 [0.7183242  0.55469126]]


,patient_id,CE,LAA
0,006388,0.689040,0.442307
1,008e5c,0.705428,0.580366
2,00c058,0.731945,0.697593
3,01adc5,0.718324,0.554691


In [37]:
# pd.read_csv("submission.csv")